
include a dynamic lib?


In [ ]:
const {
  equal
} = require('assert')
const {
  load,
  DataType,
  open,
  close,
  arrayConstructor,
  define
} = require('ffi-rs')
const path = require('path')
const {platform} = require('os')
const a = 1
const b = 100
const dynamicLib = platform === 'win32' ? '../sog.dll' : (platform === 'linux' ? '../sog.so' : '../sog.dylib')
// First open dynamic library with key for close
const IGNORE_PROGRAMS = [
  'Window Server',
  'Macs Fan Control',
  'Control Center',
  'Spotlight',
  'Dock',
]
function enumWindows() {
  // It only needs to be opened once.
  open({
    library: 'sog', // key
    path: path.join(__dirname, dynamicLib) // path
  })
  const count = load({
    library: "sog", // path to the dynamic library file
    funcName: 'enumWindows', // the name of the function to call
    retType: DataType.I32, // the return value type
    paramsType: [], // the parameter types
    paramsValue: [] // the actual parameter values
    // freeResultMemory: true, // whether or not need to free the result of return value memory automatically, default is false
  })
  let windows = []
  for(let i = 0; i < count; i++) {
    const result = load({
      library: "sog", // path to the dynamic library file
      funcName: 'getTitle', // the name of the function to call
      retType: DataType.String, // the return value type
      paramsType: [DataType.I32], // the parameter types
      paramsValue: [i] // the actual parameter values
      // freeResultMemory: true, // whether or not need to free the result of return value memory automatically, default is false
    })
    windows.push(result.split(/:\s/g)[1])
  }
  // Release library memory when you're not using it.
  close('libsum')
  return windows.filter((a, i, arr) => arr.indexOf(a) == i && !IGNORE_PROGRAMS.includes(a))
}

module.exports = enumWindows


In [ ]:

// Use define function to define a function signature
const res = define({
  sum: {
      library: "libsum",
      retType: DataType.I32,
      paramsType: [DataType.I32, DataType.I32],
  },
  atoi: {
      library: "libnative",
      retType: DataType.I32,
      paramsType: [DataType.String],
  }
})
equal(res.sum([1, 2]), 3)
equal(res.atoi(["1000"]), 1000)
